In [1]:
import warnings
warnings.filterwarnings("ignore")
import acquire

# tabular manipulation:
import numpy as np
import pandas as pd
# visualization:
import matplotlib.pyplot as plt
import seaborn as sns
# sklearn for scaling and clustering:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
# environment:
from env import host, user, password

def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

url = get_db_url("grocery_db")

sql = """
select *
from grocery_customers
"""

df = pd.read_sql(sql, url, index_col="customer_id")
df.head(3)

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
customer_id,,,,,,,,
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844


### Use DBSCAN to detect anomalies in other products from the customers dataset. The database name is grocery_db

In [2]:
# select what features we wish to examine
selected_feats = ['Frozen', 'Detergents_Paper', 'Delicassen']

In [3]:
# create a scaler
minmax = MinMaxScaler()

In [4]:
scaled_features = minmax.fit_transform(df[selected_feats])

In [5]:
# examine our numpy array
scaled_features

array([[0.0031063 , 0.0654272 , 0.02784731],
       [0.02854842, 0.08058985, 0.03698373],
       [0.03911643, 0.08605232, 0.16355861],
       ...,
       [0.00677142, 0.36346267, 0.03888194],
       [0.01664914, 0.00404174, 0.04426366],
       [0.00065742, 0.01161082, 0.00102211]])

In [6]:
# make sure the type is numpy so that it is suitable for feeding into our DBSCAN model
type(scaled_features)

numpy.ndarray

In [7]:
# scaled information into df
scaled_cols = [col + '_scaled' for col in selected_feats]

In [8]:
scaled_df = pd.DataFrame(scaled_features, index = df.index, columns = scaled_cols)

In [9]:
scaled_df

,Frozen_scaled,Detergents_Paper_scaled,Delicassen_scaled
customer_id,,,
0,0.003106,0.065427,0.027847
1,0.028548,0.080590,0.036984
2,0.039116,0.086052,0.163559
3,0.104842,0.012346,0.037234
4,0.063934,0.043455,0.108093
...,...,...,...
435,0.215469,0.004385,0.045912
436,0.073713,0.002205,0.048874
437,0.006771,0.363463,0.038882


In [10]:
df = df.merge(scaled_df, on=df.index)

In [11]:
df.head(3)

,key_0,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Frozen_scaled,Detergents_Paper_scaled,Delicassen_scaled
0,0,2,3,12669,9656,7561,214,2674,1338,0.003106,0.065427,0.027847
1,1,2,3,7057,9810,9568,1762,3293,1776,0.028548,0.080590,0.036984
2,2,2,3,6353,8808,7684,2405,3516,7844,0.039116,0.086052,0.163559


In [12]:
# Create the DBSCAN model:
# make it, fit it, use it

In [13]:
df.shape

(440, 12)

In [14]:
# make it
dbsc = DBSCAN(eps = 0.1, min_samples=20)

In [15]:
# fit it
dbsc.fit(scaled_features)

DBSCAN(eps=0.1, min_samples=20)

In [16]:
# use it
dbsc.labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,  0,
        0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -1, -1, -1,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [17]:
df['labels'] = dbsc.labels_

In [18]:
df[selected_feats + ['labels']].head(3)

,Frozen,Detergents_Paper,Delicassen,labels
0,214,2674,1338,0
1,1762,3293,1776,0
2,2405,3516,7844,0


In [19]:
df.labels.value_counts()

 0    423
-1     17
Name: labels, dtype: int64

In [20]:
# We have our cluster labels now:
# Clusters: 1 (label: 0)
# OUtliers: (Label: -1)

In [21]:
# Explore

In [22]:
sns.scatterplot(x = 'Frozen', y = 'Detergents_Paper', hue='labels', data=df)

<AxesSubplot:xlabel='Frozen', ylabel='Detergents_Paper'>

In [23]:
sns.scatterplot(x = 'Detergents_Paper', y = 'Delicassen', hue='labels', data=df)

<AxesSubplot:xlabel='Detergents_Paper', ylabel='Delicassen'>

In [24]:
# 3D scale

fig = plt.figure(1, figsize=(10, 10))
ax = Axes3D(fig)

# plot the points
ax.scatter(df.Frozen,
          df.Detergents_Paper,
          df.Delicassen,
          c=df.labels,
          edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

ax.set_xlabel('Frozen')
ax.set_ylabel('Detergents_Paper')
ax.set_zlabel('Delicassen')

Text(0.5, 0, 'Delicassen')

### Use DBSCAN to detect anomalies in requests from the curriculum access logs in the curriculum_logs database.

In [36]:
# Import .csv file and convert it to a DataFrame object
df = pd.read_table("anonymized-curriculum-access-07-2021.txt", sep = '\s', header = None, 
                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

df.head()

,date,time,page,id,cohort,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


### Explore

In [37]:
id_counts = df.groupby(['id'])['date','cohort','page'].nunique()
cohort_counts = df.groupby(['cohort'])['page','id','date'].nunique()
first_access = df.groupby(['id'])['date'].min()
id_by_first_access_date = pd.DataFrame({'first_access_date': first_access}).reset_index().groupby('first_access_date').count()

In [38]:
id_by_first_access_date

,id
first_access_date,
2018-01-26,50
2018-01-27,2
2018-01-28,2
2018-01-29,12
2018-01-30,7
...,...
2021-06-28,23
2021-07-04,2
2021-07-06,2


In [39]:
fig = plt.figure(figsize=(16, 8))
plt.plot(id_by_first_access_date)
plt.xticks(rotation = 90)
plt.show()

In [40]:
id_counts[id_counts.cohort > 1]

,date,cohort,page
id,,,
1,685,2,1018
11,896,2,883
25,114,2,154
37,43,2,156
40,102,2,120
41,221,2,186
51,99,2,239
53,815,2,443
64,765,3,453


Could someone be stealing the content of our curriculum for their benefit beyond personal education? If so, we would probably see them accessing a large number of unique pages. I would imagine they wouldn't spend much time on each page, maybe taking screen shots, copy/paste or downloading the content. Let's take a look.
Aggregate and compute 2 features...number of unique pages and total page views.

In [41]:
df.head()

,date,time,page,id,cohort,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [42]:
page_views = df.groupby(['id'])['page'].agg(['count','nunique'])
page_views

,count,nunique
id,,
1,8190,1018
2,1541,151
3,1562,169
4,692,143
5,1701,197
...,...,...
1078,44,12
1079,60,10
1080,27,9


Scale each attribute lineraly

In [43]:
scaler = MinMaxScaler().fit(page_views)
page_views_scaled_array = scaler.transform(page_views)
page_views_scaled_array[0:10]

array([[0.41799806, 1.        ],
       [0.07860752, 0.14749263],
       [0.07967944, 0.16519174],
       [0.0352713 , 0.13962635],
       [0.08677454, 0.1927237 ],
       [0.07181869, 0.22025565],
       [0.09198101, 0.16715831],
       [0.02460313, 0.08751229],
       [0.02980961, 0.1425762 ],
       [0.04701138, 0.14060964]])

Construct a DBSCAN object that requires a minimum of 20 data points in a neighborhood of radius 0.1 to be considered a core point.


In [44]:
dbsc = DBSCAN(eps = .10, min_samples = 20).fit(page_views_scaled_array)

In [45]:
# Now, let's add the scaled value columns back onto the dataframe
columns = list(page_views.columns)
scaled_columns = ["scaled_" + column for column in columns]
scaled_columns

['scaled_count', 'scaled_nunique']

In [46]:
# Create a dataframe containing the scaled values
scaled_df = pd.DataFrame(page_views_scaled_array, columns=scaled_columns)
scaled_df

,scaled_count,scaled_nunique
0,0.417998,1.000000
1,0.078608,0.147493
2,0.079679,0.165192
3,0.035271,0.139626
4,0.086775,0.192724
...,...,...
1077,0.002195,0.010816
1078,0.003012,0.008850
1079,0.001327,0.007866
1080,0.001787,0.021632


In [47]:
# Merge the scaled and non-scaled values into one dataframe
page_views = page_views.merge(scaled_df, on=page_views.index)
page_views

,key_0,count,nunique,scaled_count,scaled_nunique
0,1,8190,1018,0.417998,1.000000
1,2,1541,151,0.078608,0.147493
2,3,1562,169,0.079679,0.165192
3,4,692,143,0.035271,0.139626
4,5,1701,197,0.086775,0.192724
...,...,...,...,...,...
1077,1078,44,12,0.002195,0.010816
1078,1079,60,10,0.003012,0.008850
1079,1080,27,9,0.001327,0.007866
1080,1081,36,23,0.001787,0.021632


In [48]:
page_views = page_views.drop(columns=['key_0'])
page_views.head()

,count,nunique,scaled_count,scaled_nunique
0,8190,1018,0.417998,1.000000
1,1541,151,0.078608,0.147493
2,1562,169,0.079679,0.165192
3,692,143,0.035271,0.139626
4,1701,197,0.086775,0.192724


In [49]:
page_views.head()

,count,nunique,scaled_count,scaled_nunique
0,8190,1018,0.417998,1.000000
1,1541,151,0.078608,0.147493
2,1562,169,0.079679,0.165192
3,692,143,0.035271,0.139626
4,1701,197,0.086775,0.192724


Next, we can extract our cluster labels and outliers to plot our results.

In [55]:
labels = dbsc.labels_
labels[0:10]

array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [56]:
page_views['labels'] = labels
page_views.labels.value_counts()

 0    1074
-1       8
Name: labels, dtype: int64

In [57]:
page_views[page_views.labels==-1]

,count,nunique,scaled_count,scaled_nunique,labels
0,8190,1018,0.417998,1.000000,-1
10,19592,883,1.000000,0.867257,-1
52,12822,443,0.654433,0.434612,-1
63,16559,453,0.845184,0.444444,-1
247,5371,628,0.274105,0.616519,-1
313,7783,225,0.397223,0.220256,-1
494,8765,260,0.447348,0.254671,-1
580,8080,364,0.412383,0.356932,-1


In [58]:
# Let's look at the descriptive stats for the entire population, the inliers, then the outliers/anomalies
print("Population")
print(page_views.describe())
print("-------------")
print("Inliers")
print(page_views[page_views.labels==0].describe())
print("-------------")
print("Outliers")
print(page_views[page_views.labels==-1].describe())

Population
              count      nunique  scaled_count  scaled_nunique       labels
count   1082.000000  1082.000000   1082.000000     1082.000000  1082.000000
mean     941.597967   120.389094      0.048012        0.117393    -0.007394
std     1238.102570    80.299527      0.063198        0.078957     0.085708
min        1.000000     1.000000      0.000000        0.000000    -1.000000
25%      199.000000    55.250000      0.010107        0.053343     0.000000
50%      720.500000   140.000000      0.036726        0.136676     0.000000
75%     1317.750000   165.000000      0.067212        0.161259     0.000000
max    19592.000000  1018.000000      1.000000        1.000000     0.000000
-------------
Inliers
             count      nunique  scaled_count  scaled_nunique  labels
count  1074.000000  1074.000000   1074.000000     1074.000000  1074.0
mean    867.455307   117.306331      0.044227        0.114362     0.0
std     800.249247    68.335681      0.040848        0.067193     0.0
min

In [59]:
plt.scatter(page_views['count'], page_views['nunique'], c=page_views.labels)
plt.show()